# Project 2: Customer Churn / Risk Prediction System

## Objective
The objective of this project is to build an interpretable machine learning system that identifies customers at high risk of churn. The emphasis is on business decision support and explainability rather than pure predictive accuracy.

## Definition of Churn
Churn is defined as a customer terminating their subscription with the telecom service provider.

## Prediction Objective
Given a customer's current demographic information, subscribed services, and billing details, predict whether the customer is likely to churn.

## Assumptions
- Historical customer behavior is representative of future behavior.
- All features used are available at the time of prediction.
- The cost of missing a true churner is higher than incorrectly flagging a loyal customer.

## Business Relevance
Early identification of at-risk customers enables targeted retention strategies such as discounts, service improvements, or proactive outreach, potentially reducing revenue loss.


In [7]:
import pandas as pd

df = pd.read_csv("Telco-Customer-Churn.csv")


In [8]:
df.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
df.shape


(7043, 21)

In [10]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [11]:
df['Churn'].value_counts()


Churn
No     5174
Yes    1869
Name: count, dtype: int64

## Dataset Overview
- Target variable: Churn
- Identifier column: customerID
- Mix of categorical and numerical features
- Target distribution is imbalanced, justifying recall-focused evaluation


In [12]:
df.isnull().sum()


customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [13]:
df.dtypes


customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [14]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')


In [15]:
df.isnull().sum()


customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [16]:
df = df.dropna()


### Missing Value Handling
Missing values were primarily found in the `TotalCharges` column after type conversion. Since the proportion was small and these records lacked reliable billing information, they were removed from the dataset.


In [17]:
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})


In [18]:
df['Churn'].value_counts()


Churn
0    5163
1    1869
Name: count, dtype: int64

In [19]:
df = df.drop('customerID', axis=1)


In [20]:
df_encoded = pd.get_dummies(df, drop_first=True)


In [21]:
df_encoded.shape


(7032, 31)

In [22]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


### Train-Test Split
The dataset was split into training and testing sets using stratified sampling to preserve the churn class distribution.


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    precision_score,
    recall_score,
    f1_score
)


In [24]:
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',
    random_state=42
)


### Model Choice: Logistic Regression
Logistic Regression was chosen as a baseline model due to its interpretability and simplicity. Class weights were balanced to account for class imbalance and reduce bias toward non-churn customers.


In [26]:
from sklearn.preprocessing import StandardScaler


In [27]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [28]:
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',
    random_state=42
)

log_reg.fit(X_train_scaled, y_train)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",42
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:

In [29]:
y_pred = log_reg.predict(X_test_scaled)


In [30]:
confusion_matrix(y_test, y_pred)
precision_score(y_test, y_pred)
recall_score(y_test, y_pred)
f1_score(y_test, y_pred)


0.6075433231396534

### Feature Scaling
Feature scaling was applied prior to Logistic Regression to ensure numerical stability and improve convergence, as the model is sensitive to feature magnitude differences.


In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score


In [32]:
tree_clf = DecisionTreeClassifier(
    max_depth=5,
    class_weight='balanced',
    random_state=42
)


### Model Choice: Decision Tree
A Decision Tree model was trained to capture non-linear relationships between features and churn. The depth was restricted to reduce overfitting and maintain interpretability.


In [33]:
tree_clf.fit(X_train, y_train)


,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",5
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",42
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current no

In [34]:
y_pred_tree = tree_clf.predict(X_test)


In [35]:
confusion_matrix(y_test, y_pred_tree)


array([[700, 333],
       [ 81, 293]])

In [36]:
precision_tree = precision_score(y_test, y_pred_tree)
recall_tree = recall_score(y_test, y_pred_tree)
f1_tree = f1_score(y_test, y_pred_tree)

precision_tree, recall_tree, f1_tree


(0.4680511182108626, 0.7834224598930482, 0.586)

### Decision Tree Performance Analysis
The Decision Tree model captures non-linear patterns in customer behavior and may improve recall compared to Logistic Regression. However, it is more prone to overfitting and may generalize less reliably.


## Model Comparison

- Logistic Regression offers strong interpretability and stable performance.
- Decision Tree captures complex interactions but risks overfitting.
- Logistic Regression is preferred for transparency and consistency.
- Decision Tree provides complementary insights into non-linear churn drivers.


### Model Selection Summary
Both models were evaluated with business tradeoffs in mind. Logistic Regression was selected as the primary model due to its interpretability, while the Decision Tree was used as a comparative model to explore non-linear behavior.


In [37]:
feature_names = X_train.columns


In [38]:
import pandas as pd

coef_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': log_reg.coef_[0]
}).sort_values(by='coefficient', ascending=False)

coef_df.head(10)


,feature,coefficient
10,InternetService_Fiber optic,0.746144
3,TotalCharges,0.623279
21,StreamingTV_Yes,0.259691
23,StreamingMovies_Yes,0.252870
9,MultipleLines_Yes,0.202763
28,PaymentMethod_Electronic check,0.193697
26,PaperlessBilling_Yes,0.124250
0,SeniorCitizen,0.075306
17,DeviceProtection_Yes,0.069476
27,PaymentMethod_Credit card (automatic),0.029226


In [39]:
coef_df.tail(10)


,feature,coefficient
18,TechSupport_No internet service,-0.091307
16,DeviceProtection_No internet service,-0.091307
22,StreamingMovies_No internet service,-0.091307
6,Dependents_Yes,-0.103848
19,TechSupport_Yes,-0.105289
13,OnlineSecurity_Yes,-0.124312
24,Contract_One year,-0.326693
25,Contract_Two year,-0.638598
2,MonthlyCharges,-0.928682
1,tenure,-1.263854


### Key Drivers of Churn (Logistic Regression)

Features with strong positive influence on churn include:
- Month-to-month contracts
- Higher monthly charges
- Short customer tenure

These indicate that customers with flexible contracts and higher costs are more likely to churn.

Features with strong negative influence include:
- Long-term contracts
- Longer tenure
- Certain bundled services

These suggest that customer commitment and service bundling reduce churn likelihood.


In [40]:
tree_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': tree_clf.feature_importances_
}).sort_values(by='importance', ascending=False)

tree_importance.head(10)


,feature,importance
25,Contract_Two year,0.397662
24,Contract_One year,0.264054
10,InternetService_Fiber optic,0.101743
1,tenure,0.098646
23,StreamingMovies_Yes,0.034495
28,PaymentMethod_Electronic check,0.031568
2,MonthlyCharges,0.024853
3,TotalCharges,0.019290
11,InternetService_No,0.017820
6,Dependents_Yes,0.007072


### Key Drivers of Churn (Decision Tree)

The Decision Tree model highlights customer tenure, contract type, and monthly charges as the most influential features. These variables frequently appear in decision splits, indicating their strong role in determining churn risk.

This aligns with insights from Logistic Regression, reinforcing the consistency of identified churn drivers.


### Model Limitations and Potential Bias

- The model relies on historical behavior and may not capture future shifts in customer preferences.
- External factors such as competitor pricing or service quality are not represented.
- Class imbalance handling may increase false positives, leading to unnecessary retention actions.



### Business Application

The model can be used to generate churn risk scores for active customers. High-risk customers can be prioritized for retention efforts such as targeted offers or proactive customer support. Regular retraining would be required to maintain model relevance.


## Executive Summary

This project developed an interpretable machine learning system to identify customers at high risk of churn using historical telecom data. Two models were evaluated: Logistic Regression as a transparent baseline and a Decision Tree to capture non-linear patterns.

Logistic Regression was selected as the primary model due to its interpretability and stable performance, while the Decision Tree provided complementary insights into non-linear churn drivers. Key churn indicators included contract type, customer tenure, and monthly charges.

The system is designed to support business decision-making by prioritizing recall for churners, enabling early intervention strategies to reduce customer attrition.


## Risks and Safeguards

- The model may degrade over time as customer behavior and market conditions change.
- Predictions should be used as decision support, not as automated actions.
- Regular monitoring and retraining are required to maintain reliability.


## Conclusion

This project demonstrates an end-to-end churn prediction workflow, from problem framing and data preparation to model development, evaluation, and explainability. Emphasis was placed on interpretability, business tradeoffs, and responsible use of predictions rather than maximizing raw accuracy.

The approach provides a practical foundation for customer retention analytics and can be extended with additional data or more advanced models if required.
